In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import os
import random
from sklearn.preprocessing import LabelEncoder


def train_Ela_Model(dataset_path, lr, ep):
    def convert_to_ela_image(path, quality):
        filename = path
        resaved_filename = filename.split('.')[0] + '.resaved.jpg'
        im = Image.open(filename).convert('RGB')
        im.save(resaved_filename, 'JPEG', quality=quality)
        resaved_im = Image.open(resaved_filename)
        ela_im = ImageChops.difference(im, resaved_im)
        extrema = ela_im.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1
        scale = 255.0 / max_diff

        ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
        return ela_im

    X = []
    Y = []

    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            X.append(np.array(convert_to_ela_image(image_path, 90).resize((128, 128))) / 255.0)  # Removed flatten
            label_encoder = LabelEncoder()
            Y.append(label_encoder.fit_transform([class_folder]))  # Assuming class folders represent different classes

    X = np.array(X)
    Y = to_categorical(Y, num_classes=len(os.listdir(dataset_path)))
    X = X.reshape(-1, 128, 128, 3)

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=5, shuffle=True)

    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=(128, 128, 3)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
    model.add(MaxPool2D(pool_size=2, strides=None, padding='valid', data_format='channels_last'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.50))
    model.add(Dense(len(os.listdir(dataset_path)), activation="softmax"))
    model.summary()

    optimizer = RMSprop(learning_rate=lr, rho=0.9, epsilon=1e-08)

    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    epochs = ep
    batch_size = 5

    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val), verbose=2)

    # Rest of your code for plotting and saving the model

    return history  # Optionally, you might want to return the training history for further analysis

# Example usage:
dataset_path = '/content/drive/MyDrive/CASIA2.0_Groundtruth'
learning_rate = 0.001
num_epochs = 10
train_Ela_Model(dataset_path, learning_rate, num_epochs)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 30, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 30, 30, 32)        0         
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 256)               7373056   
                                                        

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


924/924 - 274s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 274s/epoch - 297ms/step
Epoch 2/10
924/924 - 280s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 280s/epoch - 303ms/step
Epoch 3/10
924/924 - 273s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 273s/epoch - 296ms/step
Epoch 4/10
924/924 - 280s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 280s/epoch - 303ms/step
Epoch 5/10
924/924 - 274s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 274s/epoch - 297ms/step
Epoch 6/10
924/924 - 272s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 272s/epoch - 294ms/step
Epoch 7/10
924/924 - 279s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 279s/epoch - 302ms/step
Epoch 8/10
924/924 - 273s - loss: 0.0000e+00 - accuracy: 1.